# PyEZ study note

# connect to the device

In [3]:
from jnpr.junos import Device

with Device(host='10.85.160.211',user='labroot',password='lab123') as dev:
    print(dev.facts)

{'2RE': False, 'HOME': '/var/home/labroot', 'RE0': {'mastership_state': 'master', 'status': 'OK', 'model': 'RE-QFX5110-48S-4C', 'last_reboot_reason': '0x1:power cycle/failure', 'up_time': '3 days, 10 hours, 48 minutes, 23 seconds'}, 'RE1': None, 'RE_hw_mi': False, 'current_re': ['master', 'node', 'fwdd', 'member', 'pfem', 're0', 'localre'], 'domain': 'ultralab.juniper.net', 'fqdn': 'jtac-QFX5110-48S-4C-r064.ultralab.juniper.net', 'hostname': 'jtac-QFX5110-48S-4C-r064', 'hostname_info': {'localre': 'jtac-QFX5110-48S-4C-r064'}, 'ifd_style': 'SWITCH', 'junos_info': {'localre': {'text': '21.2I-18518-TB-201988-_cd-builder', 'object': junos.version_info(major=(21, 2), type=I, minor=-18518-TB-201988-_cd-builder, build=None)}}, 'master': 'RE0', 'model': 'QFX5110-48S-4C', 'model_info': {'localre': 'QFX5110-48S-4C'}, 'personality': 'SWITCH', 're_info': {'default': {'0': {'mastership_state': 'master', 'status': 'OK', 'model': 'RE-QFX5110-48S-4C', 'last_reboot_reason': '0x1:power cycle/failure'}, 

# Mapping cli command with RPC method

PyEZ supports low level capabilities to allow user to invoke XML RPCs in Python

Each cli command could be mapped with a XML RPC.

There are multiple ways to get the RPC method based on the cli command.

1.From Junos cli, adding "| display xml rpc" after each cli command

labroot@grapevine-re0> show version | display xml rpc 
<rpc-reply xmlns:junos="http://xml.juniper.net/junos/17.4R3/junos">
    <rpc>
        <get-software-information>  <<<<<<< this is the rpc method which could be used in pyez.
        </get-software-information>
    </rpc>
    <cli>
        <banner></banner>
    </cli>
</rpc-reply>
    
2.use PyEZ function  display_xml_rpc('<cli command>')
    


In [6]:
from jnpr.junos import Device

with Device(host='10.85.160.211',user='labroot',password='lab123') as dev:
    print(dev.display_xml_rpc('show version'))

<Element get-software-information at 0x10d76fd20>


# Pull data from Junos device via PyEZ

In [12]:
#We could still interact with juniper device directly with cli command.  
from jnpr.junos import Device

with Device(host='10.85.160.211',user='labroot',password='lab123') as dev:
    junos_version=dev.cli('show version')
    #response of dev.cli() is string, exactly the same as issue the cli command in Junos prompt.
    print(type(junos_version))
    print(junos_version)
    #There will be a RuntimeWarning popping out, functions from warning library is not able to filter it out. 
    #Need to add "warning=False" in the cli() method.
    junos_version=dev.cli('show version',warning=False)
    #Note, cli() does not give out warning.

/opt/anaconda3/lib/python3.7/site-packages/jnpr/junos/device.py:694: RuntimeWarning: 
CLI command is for debug use only!
Instead of:
cli('show version')
Use:
rpc.get_software_information()

  warnings.warn(warning_string, RuntimeWarning)


<class 'str'>

localre:
--------------------------------------------------------------------------
Hostname: jtac-QFX5110-48S-4C-r064
Model: qfx5110-48s-4c
Junos: 21.2I-18518-TB-201988-_cd-builder
JUNOS OS Kernel 64-bit  [20201211.040915__ci_fbsd_builder_stable_11]
JUNOS OS libs [20201211.040915__ci_fbsd_builder_stable_11]
JUNOS OS runtime [20201211.040915__ci_fbsd_builder_stable_11]
JUNOS OS time zone information [20201211.040915__ci_fbsd_builder_stable_11]
JUNOS OS libs compat32 [20201211.040915__ci_fbsd_builder_stable_11]
JUNOS OS 32-bit compatibility [20201211.040915__ci_fbsd_builder_stable_11]
JUNOS py extensions [20201221.103940__cd-builder_dev_common]
JUNOS py base [20201221.103940__cd-builder_dev_common]
JUNOS OS vmguest [20201211.040915__ci_fbsd_builder_stable_11]
JUNOS OS crypto [20201211.040915__ci_fbsd_builder_stable_11]
JUNOS OS boot-ve files [20201211.040915__ci_fbsd_builder_stable_11]
JUNOS network stack and utilities [20201221.103940__cd-builder_dev_common]
JUNOS libs [

In [17]:
#Another method to pull data is via shell. 
#import StartShell class to interact with shell directly.
#All the PFE commands, Linux command, could be collected in this method.

from jnpr.junos import Device
from jnpr.junos.utils.start_shell import StartShell
from pprint import pprint


with Device(host='10.85.160.211',user='labroot',password='lab123') as dev:
    ss=StartShell(dev)
    ss.open()
    fpc=ss.run('cli -c "show chassis fpc"')
    ss.close()
    
#Response is a Tuple
print(type(fpc))
pprint(fpc)
    

<class 'tuple'>
(True,
 'cli -c "show chassis fpc"\r'
 '\r\n'
 '                     Temp  CPU Utilization (%)   CPU Utilization (%)  '
 'Memory    Utilization (%)\r\n'
 'Slot State            (C)  Total  Interrupt      1min   5min   15min  DRAM '
 '(MB) Heap     Buffer\r\n'
 '  0  Online            35      4          0        4      4      4    '
 '16384      27         49\r\n'
 '  1  Empty           \r\n'
 '  2  Empty           \r\n'
 '  3  Empty           \r\n'
 '  4  Empty           \r\n'
 '  5  Empty           \r\n'
 '  6  Empty           \r\n'
 '  7  Empty           \r\n'
 '  8  Empty           \r\n'
 '  9  Empty           \r\n'
 '% ')


In [34]:
#Third Method is to do the job via XML RPC.

from jnpr.junos import Device
from jnpr.junos.utils.start_shell import StartShell
from lxml import etree
from pprint import pprint


with Device(host='10.85.160.211',user='labroot',password='lab123') as dev:
    #Use RPC method 
    fpc_xml=dev.rpc.get_fpc_information()
    #response is a lxml.etree element
    #Need to import etree from lxml to interpret the content
    
    print(type(fpc_xml))
    pprint(etree.dump(fpc_xml))
    
    #There are several methods which could be used to iterabing the XML content:
        
        #findall(): find all matching blocks based on the input xpath
        #findtext(): return the content based on the input xpath
        
    for fpc in fpc_xml.findall('./fpc'):
        slot=fpc.findtext('slot')
        cpu_15min=fpc.findtext('cpu-15min-avg')
        print(f'Slot: {slot}  cpu-15min-avg:{cpu_15min} ')
    
    
    #the response data type could be converted into json format or text format:
    fpc_text=dev.rpc.get_fpc_information({'format':'text'})
    #RPC response is still a lxml.etree element, but the content is in plain text instead of XML format
    print(type(fpc_text))
    print(etree.dump(fpc_text))
    
    #Response is a python native dict.
    fpc_json=dev.rpc.get_fpc_information({'format':'json'})
    print(type(fpc_json))
    pprint(fpc_json)


<class 'lxml.etree._Element'>
<fpc-information style="brief">
  <fpc>
    <slot>0</slot>
    <state>Online</state>
    <temperature celsius="35">35</temperature>
    <cpu-total>4</cpu-total>
    <cpu-interrupt>0</cpu-interrupt>
    <cpu-1min-avg>5</cpu-1min-avg>
    <cpu-5min-avg>4</cpu-5min-avg>
    <cpu-15min-avg>4</cpu-15min-avg>
    <memory-dram-size>16384</memory-dram-size>
    <memory-heap-utilization>27</memory-heap-utilization>
    <memory-buffer-utilization>49</memory-buffer-utilization>
  </fpc>
  <fpc>
    <slot>1</slot>
    <state>Empty</state>
  </fpc>
  <fpc>
    <slot>2</slot>
    <state>Empty</state>
  </fpc>
  <fpc>
    <slot>3</slot>
    <state>Empty</state>
  </fpc>
  <fpc>
    <slot>4</slot>
    <state>Empty</state>
  </fpc>
  <fpc>
    <slot>5</slot>
    <state>Empty</state>
  </fpc>
  <fpc>
    <slot>6</slot>
    <state>Empty</state>
  </fpc>
  <fpc>
    <slot>7</slot>
    <state>Empty</state>
  </fpc>
  <fpc>
    <slot>8</slot>
    <state>Empty</state>
  </fpc>
 

# PyEZ table and view

Instead of iterating the RPC response, PyEZ offers "Table and Views" feature to invoke RPC and map the response into a Python data structure.

Tables and Views are YML file which contains the desired RPC method and specific fields for matching.

There are pre-defined Tables and we could build our own.

## Use pre-defined Tables

Before using the pre-defined tables, we need to find the location of these files.

These files should locate in the pyez directory:

(base) muzhang-mbp:op muzhang$ pwd
/opt/anaconda3/lib/python3.7/site-packages/jnpr/junos/op
(base) muzhang-mbp:op muzhang$ ls *.yml
arp.yml				intopticdiag.yml		phyport.yml
bfd.yml				inventory.yml			ppm.yml
bgp.yml				isis.yml			routes.yml
ccc.yml				l2circuit.yml			securityzone.yml
ddos.yml			lacp.yml			systemstorage.yml
elsethernetswitchingtable.yml	ldp.yml				taskmemory.yml
ethernetswitchingtable.yml	lldp.yml			teddb.yml
ethport.yml			nd.yml				vlan.yml
fpc.yml				ospf.yml			xcvr.yml
idpattacks.yml			pfestats.yml


Check all available tables:

(base) muzhang-mbp:op muzhang$ grep Table: *.yml
arp.yml:ArpTable:
bfd.yml:BfdSessionTable:
bfd.yml:_BfdSessionClientTable:
bgp.yml:bgpTable:
ccc.yml:CCCTable:
ddos.yml:DDOSTable:
elsethernetswitchingtable.yml:ElsEthernetSwitchingTable:
ethernetswitchingtable.yml:EthernetSwitchingTable:
ethernetswitchingtable.yml:_MacTableEntriesTable:
ethernetswitchingtable.yml:_MacTableInterfacesTable:
ethport.yml:EthPortTable:
fpc.yml:FpcHwTable:
fpc.yml:FpcMiReHwTable:
fpc.yml:FpcInfoTable:
fpc.yml:FpcMiReInfoTable:
idpattacks.yml:IDPAttackTable:
intopticdiag.yml:PhyPortDiagTable:
inventory.yml:ModuleTable:
isis.yml:IsisAdjacencyTable:
isis.yml:_IsisAdjacencyLogTable:
l2circuit.yml:L2CircuitConnectionTable:
lacp.yml:LacpPortTable:
lacp.yml:_LacpPortStateTable:
lacp.yml:_LacpPortProtoTable:
ldp.yml:LdpNeighborTable:
ldp.yml:_LdpNeighborHelloFlagsTable:
ldp.yml:_LdpNeighborTypesTable:
lldp.yml:LLDPNeighborTable:
nd.yml:NdTable:
ospf.yml:OspfNeighborTable:
ospf.yml:OspfInterfaceTable:
ospf.yml:ospfTable:
ospf.yml:OspfRoutesTable:
ospf.yml:OSPFStatsTable:
ospf.yml:OSPFIOStatsTable:
pfestats.yml:PFEStatsTrafficTable:
pfestats.yml:_PFEStatsLocalTrafficTable:
pfestats.yml:_PFEStatsLocalProtocolTable:
phyport.yml:PhyPortTable:
phyport.yml:PhyPortStatsTable:
phyport.yml:PhyPortErrorTable:
ppm.yml:PPMTable:
routes.yml:RouteTable:
routes.yml:RouteSummaryTable:
routes.yml:_rspTable:
securityzone.yml:SecurityZoneTable:
securityzone.yml:_szInterfaceTable:
systemstorage.yml:SystemStorageTable:
taskmemory.yml:TaskMemoryTable:
teddb.yml:TedTable:
teddb.yml:_linkTable:
teddb.yml:TedSummaryTable:
vlan.yml:VlanTable:
xcvr.yml:XcvrTable:


All above tables could be used directly in Python

In [37]:
from jnpr.junos import Device

from jnpr.junos.op.fpc import FpcInfoTable
from pprint import pprint


with Device(host='10.85.160.211',user='labroot',password='lab123') as dev:
    #one FPCinfoTable object is initiated.
    fpc_table=FpcInfoTable(dev)
    
    fpc_table.get()
    
    print(fpc_table)
    pprint(fpc_table.items())
    

FpcInfoTable:10.85.160.211: 10 items
[('0', [('state', 'Online'), ('memory', '27'), ('cpu', '4')]),
 ('1', [('state', 'Empty'), ('memory', None), ('cpu', None)]),
 ('2', [('state', 'Empty'), ('memory', None), ('cpu', None)]),
 ('3', [('state', 'Empty'), ('memory', None), ('cpu', None)]),
 ('4', [('state', 'Empty'), ('memory', None), ('cpu', None)]),
 ('5', [('state', 'Empty'), ('memory', None), ('cpu', None)]),
 ('6', [('state', 'Empty'), ('memory', None), ('cpu', None)]),
 ('7', [('state', 'Empty'), ('memory', None), ('cpu', None)]),
 ('8', [('state', 'Empty'), ('memory', None), ('cpu', None)]),
 ('9', [('state', 'Empty'), ('memory', None), ('cpu', None)])]


This is the content of the Table YML file:

# -------------------------------------------------------------------
# Table
# -------------------------------------------------------------------
# retrieve the FPC status information; corresponds to:
# > show chassis fpc
# -------------------------------------------------------------------

FpcInfoTable:
    rpc: get-fpc-information
    item: .//fpc
    key: slot
    view: _fpc_info_view

FpcMiReInfoTable:
    rpc: get-fpc-information
    item: .//fpc
    key: 
        - ancestor::multi-routing-engine-item/re-name
        - slot
    view: _fpc_info_view

_fpc_info_view:
    fields:
        state: state
        memory: memory-heap-utilization
        cpu: cpu-total



We could write our own Table file if the pre-defined Table could not meet our goal.

Example:
    
---   <<<<YML file start with ---
UserFPCTable:   <<<< this is the name of the table, which will be imported later on in Python script
  rpc: get-fpc-information  <<< this is the RPC method needed to be involved,
  item: fpc   <<<< xpath, which is used to iterate to the specific blocks.
  key: slot
  view: UserFPCView  <<< View is defined with all the interested field name.

UserFPCView:
  fields:
    state: state
    cpu-total: cpu-total
    memory-heap-utilization: memory-heap-utilization
    memory-buffer-utilization: memory-buffer-utilization

In [40]:

from jnpr.junos import Device
from jnpr.junos.factory import loadyaml

from jnpr.junos.op.fpc import FpcInfoTable
from pprint import pprint


with Device(host='10.85.160.211',user='labroot',password='lab123') as dev:
    user_yml=loadyaml('./pyez_basic/UserFPCTable.yml')
    globals().update(user_yml)

    user_table=UserFPCTable(dev)
    user_table.get()
    pprint(user_table.items())

/opt/anaconda3/lib/python3.7/site-packages/jnpr/junos/factory/__init__.py:33: ResourceWarning: unclosed file <_io.TextIOWrapper name='./pyez_basic/UserFPCTable.yml' mode='r' encoding='UTF-8'>
  return FactoryLoader().load(yaml.load(open(path, 'r'), Loader=yaml.FullLoader))


NameError: name 'user_defs' is not defined